### Loading requirements

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

## P0. Read the dataset

In [3]:
bank = pd.read_csv('C:/Users/nktun/OneDrive/Documents/GitHub/Bank_Marketing/bank-additional-full.csv', delimiter=';')
bank = bank.drop('duration', axis=1) #Document said this column highly affects output y

In [4]:
bank.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


## P1. EDA

Check for null values

In [5]:
unknown_counts = (bank == 'unknown').sum()

null_df = pd.DataFrame(columns=['values', 'percentage'])
null_df['values'] = unknown_counts
null_df['percentage'] = round(unknown_counts/bank.count() * 100, 4)
# Show the df of null values
null_df

,values,percentage
age,0,0.0000
job,330,0.8012
marital,80,0.1942
education,1731,4.2027
default,8597,20.8726
housing,990,2.4036
loan,990,2.4036
contact,0,0.0000
month,0,0.0000
day_of_week,0,0.0000


The percentage of Unknown values in each column seems low (less than 5%) apart from **Default** columns. Therefore, the impact is expected to be minor

In [6]:
bank[bank['default'] == 'yes']

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
21580,48,technician,married,professional.course,yes,no,no,cellular,aug,tue,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
21581,48,technician,married,professional.course,yes,yes,no,cellular,aug,tue,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,no
24866,31,unemployed,married,high.school,yes,no,no,cellular,nov,tue,2,999,1,failure,-0.1,93.200,-42.0,4.153,5195.8,no


Only 3 records of default on loan -> Insignificant

Select columns of categorical and numerical variables for preprocessing

In [7]:
# Categorical columns with <10 unique values
categorical_cols = [cname for cname in bank.columns if bank[cname].nunique() < 10 and bank[cname].dtype == "object"] #Because this is the output
categorical_cols.remove('y')

In [8]:
bank.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,24,technician,single,professional.course,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## P1.1. Train-test split

In [9]:
# Numerical columns
numerical_cols = [cname for cname in bank.columns if bank[cname].dtype in ['int64', 'float64']]
numerical_cols

['age',
 'campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

Null values are currently given the "unknown" label. Perhaps it would be wise to just replace them with NaN and NULL

### Separate target from predictors

In [10]:
y = bank.y
X = bank.drop('y', axis=1)

### Split the data into the training set and the testing set

In [11]:
from sklearn.model_selection import train_test_split

#### The data shall be split into 2 parts: 80% for training and 20% for validation. Set random state = 0

In [12]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

!!!Always remember that the data has to be split before any preprocessing can be applied (else data leakage)

In [13]:
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

## P2. Begin preprocessing